<a href="https://colab.research.google.com/github/brunopdrocha/doctor_agent/blob/main/Agent_Medic_CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agent AI Clinical analysis

The primary objective of this work is the implementation of an ai agent designed for the purpose of diagnosing users generating a clinical analasys

In [ ]:
!python --version

Python 3.11.12


In [ ]:
# Installing CrewAI Libraries
!pip install crewai crewai-tools duckduckgo-search

In [ ]:
#Import libs to create an agent
from crewai import Crew, Process, Agent, Task, TaskOutput, CrewOutput , LLM

In [ ]:
from google.colab import userdata
import os

token    = userdata.get('GITHUB_API_KEY')            # seu PAT com models:read
endpoint = "https://models.github.ai/inference"      # raiz do GitHub Models

llm = LLM(
    model="openai/gpt-4.1",    # nome exato do modelo no GitHub Models
    base_url=endpoint,
    api_key=token,
    temperature=0.2,
    max_tokens = 32768,
    publisher="openai"
)



In [ ]:
# Try our llm model
llm.call("Qual é a capital de Brasil")

🤖 Agent: Médico
    Status: In Progress

'A capital do Brasil é **Brasília**.'

## Custom Tool Lang Chain + ScrapeWebSiteTool

We are going to use a DuckGoSearch to simulate a SerperDevTool from CrewAi

In [ ]:
# Custom Tool
from langchain_community.tools import DuckDuckGoSearchResults
from crewai_tools import ScrapeWebsiteTool
from crewai.tools import BaseTool
from pydantic import Field
import json

class SearchTool(BaseTool):
    name: str = "Search"
    description: str = "Use para buscar informações atuais em sites de notícias e blogs. Ideal para descobrir tendências ou checar fatos recentes."
    search: DuckDuckGoSearchResults = Field(default_factory=lambda: DuckDuckGoSearchResults(output_format="json", max_results=10, region='pt-br'))

    def _run(self, query: str) -> str:
        try:
            results = self.search.run(query)

            # Se for string, tenta converter
            if isinstance(results, str):
                try:
                    results = json.loads(results)
                except:
                    return results  # Retorna como está se não for JSON válido

            # Normaliza formato tipo Serper
            response = [
                {
                    "title": item.get("title"),
                    "link": item.get("link"),
                    "snippet": item.get("snippet")
                }
                for item in results if "title" in item and "link" in item
            ]

            return json.dumps(response, indent=2, ensure_ascii=False)

        except Exception as e:
            return f"Erro ao realizar a busca: {str(e)}"

In [ ]:
SearchTool().run("como melhorar a colica")

Using Tool: Search


'[\n  {\n    "title": "11 posições para aliviar cólica menstrual + vídeo - Saúde umCOMO",\n    "link": "https://saude.umcomo.com.br/artigo/posicoes-para-aliviar-colica-menstrual-30833.html",\n    "snippet": "Essa posição de yoga ajuda a aliviar não apenas as cólicas menstruais, como também as dores na lombar. Ela consiste em você deitar de barriga para cima, com os braços estendidos na altura dos ombros e, em seguida, elevar as pernas dobradas até a altura do peito e girar para a direita e depois para a esquerda. 9. Postura Supta Baddha ..."\n  },\n  {\n    "title": "Cólica menstrual exercícios que auxiliam nas dores ... - Manalinda",\n    "link": "https://www.manalinda.com/blog/colica-menstrual-exercicios-que-auxiliam-nas-dores",\n    "snippet": "Quais tipos de exercícios são mais eficazes para aliviar cólicas menstruais? Exercícios leves a moderados, como caminhada, ioga, pilates e alongamentos, são altamente eficazes para aliviar cólicas menstruais. Essas atividades ajudam a melhora

## Agent

Let's create our agent who is gonna analyse

In [ ]:
medic = Agent(
  role="Médico",
  goal="Analisar os sintomas descritos pelo paciente e gerar um laudo médico claro, estruturado e embasado em fontes confiáveis.",
  backstory="""
  Você é um médico especializado em diagnósticos preliminares com apoio de informações da internet. Seu papel é receber os sintomas relatados por um paciente e, com base neles, fazer uma análise cuidadosa.

  Para garantir precisão, você deve:
  1. Buscar na web informações médicas recentes e relevantes sobre os sintomas relatados.
  2. Validar possíveis diagnósticos ou explicações clínicas compatíveis com os sintomas.
  3. Gerar um laudo médico estruturado, com linguagem compreensível ao paciente, incluindo:
     - Descrição dos sintomas relatados.
     - Possíveis causas ou condições associadas.
     - Recomendação preliminar (ex: procurar um especialista, fazer exames).
     - Fontes ou referências utilizadas na análise.

  Utilize múltiplas buscas na web se necessário, e cite fontes confiáveis como apoio para sua análise. observação cuidado com sites bloqueados ou inexistentes isso pode conflitar com a
  sua análise fique atento com estes pontos.
  """,
  llm=llm

)


## Tasks

Let's build 2 tasks that the medic have to follow

In [ ]:
from crewai import Task

pacient_analyze = Task(
    description="""
        Analisar cuidadosamente os sintomas relatados pelo paciente e realizar uma análise médica preliminar com base nessas informações.
        Utilizar ferramentas de busca web para investigar possíveis causas, condições associadas e contextos relevantes.
    """,
    expected_output="""
        Um parecer médico preliminar sobre o tema {topic}, abordando os principais sintomas descritos pelo paciente,
        com explicações claras e objetivas. A resposta deve ser semelhante à de um médico durante uma consulta clínica,
        sendo firme, empática e informativa, sem emitir um diagnóstico definitivo.
    """,
    agent=medic,
    tools=[ScrapeWebsiteTool(), SearchTool()]
)


clinical_analysis = Task(
    description="""
        Com base nos sintomas analisados, elaborar um laudo médico estruturado e profissional.
        O laudo deve oferecer sugestões baseadas em evidências, prezando pela clareza e apoio ao paciente.

        O documento deve usar:
        - Cabeçalhos (Markdown `#`, `##`, `###`).
        - Linhas horizontais (`---`) para separar seções.
        - Listas com marcadores claros (`-` ou `*`).
        - Texto em **negrito** e _itálico_ para enfatizar pontos-chave.
        - Blocos de destaque, quando necessário, com citações Markdown (`> texto`).

    """,
    expected_output="""
        Um laudo médico contendo bem estilizado e com visual elegante contendo os seguintes pontos :
        - Resumo dos sintomas apresentados
        - Possíveis causas ou diagnósticos diferenciais
        - Indicação de medicamentos (nome, posologia, duração)
        - Recomendações clínicas e orientações de cuidado
        - Observações importantes (como evitar automedicação ou procurar atendimento emergencial)
        - Fontes utilizadas (se aplicável)


    """,
    agent=medic,
    output_file="report.md"
)

## Crew

Now let's create our crew to analyse and work for us


In [ ]:
# Callig our Crew
Crew = Crew(
    agents=[medic],
    tasks=[pacient_analyze, clinical_analysis],
    verbose=True,
    process = Process.sequential
)

## Running Crew

In [ ]:
 from datetime import datetime

 # Entrance Input
 entrada = "Ola no momento estou sentindo muita dor no abmonem ao lado direito da minha barriga o que isso pode ser"
 inputs = {
        'topic':entrada,
        'current_year': str(datetime.now().year)
    }

resultado = Crew.kickoff(inputs = inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0163a885-dcab-4f92-80ec-4cb44ad50e00                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 0d02df78-d223-4795-a4c7-7c85387cb8a7
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 0d02df78-d223-4795-a4c7-7c85387cb8a7
       Status: Executing Task...
    └── 🤖 Agent: Médico
            Status: In Progress

# Agent: Médico
## Task: 
        Analisar cuidadosamente os sintomas relatados pelo paciente e realizar uma análise médica preliminar com base nessas informações.
        Utilizar ferramentas de busca web para investigar possíveis causas, condições associadas e contextos relevantes.
    


🤖 Agent: Médico
    Status: In Progress

...

# Agent: Médico
## Task: 
        Com base nos sintomas analisados, elaborar um laudo médico estruturado e profissional.
        O laudo deve oferecer sugestões baseadas em evidências, prezando pela clareza e apoio ao paciente.

        O documento deve usar:
        - Cabeçalhos (Markdown `#`, `##`, `###`).
        - Linhas horizontais (`---`) para separar seções.
        - Listas com marcadores claros (`-` ou `*`).
        - Texto em **negrito** e _itálico_ para enfatizar pontos-chave.
        - Blocos de destaque, quando necessário, com citações Markdown (`> texto`).

    


# Agent: Médico
## Final Answer: 
# Laudo Médico Preliminar

---

## **Resumo dos Sintomas Apresentados**

Você relatou sentir **dor intensa no abdômen**, localizada ao _lado direito da barriga_. Este sintoma pode estar associado a diversas condições, algumas das quais exigem atenção médica imediata. Outros sintomas que podem estar presentes (mas não foram relatados) e que aumentam a gravidade incluem: febre, ná

🚀 Crew: crew
├── 📋 Task: 0d02df78-d223-4795-a4c7-7c85387cb8a7
│      Assigned to: Médico
│      Status: ✅ Completed
│   └── 🤖 Agent: Médico
│           Status: ✅ Completed
└── 📋 Task: bb3e5549-79fb-42c1-b599-7a6ec86445fc
       Status: Executing Task...
    └── 🤖 Agent: Médico
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 0d02df78-d223-4795-a4c7-7c85387cb8a7
│      Assigned to: Médico
│      Status: ✅ Completed
│   └── 🤖 Agent: Médico
│           Status: ✅ Completed
└── 📋 Task: bb3e5549-79fb-42c1-b599-7a6ec86445fc
       Assigned to: Médico
       Status: ✅ Completed
    └── 🤖 Agent: Médico
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: bb3e5549-79fb-42c1-b599-7a6ec86445fc                                                                     │
│  Agent: Médico                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0163a885-dcab-4f92-80ec-4cb44ad50e00                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
resultado

CrewOutput(raw='# Laudo Médico Preliminar\n\n---\n\n## **Resumo dos Sintomas Apresentados**\n\nVocê relatou sentir **dor intensa no abdômen**, localizada ao _lado direito da barriga_. Este sintoma pode estar associado a diversas condições, algumas das quais exigem atenção médica imediata. Outros sintomas que podem estar presentes (mas não foram relatados) e que aumentam a gravidade incluem: febre, náuseas, vômitos, dificuldade para urinar, sangue na urina, icterícia (pele ou olhos amarelados) e alterações no hábito intestinal.\n\n---\n\n## **Possíveis Causas ou Diagnósticos Diferenciais**\n\nCom base em informações médicas recentes e confiáveis, as principais causas para dor abdominal no lado direito incluem:\n\n- **Apendicite**  \n  > Inflamação do apêndice, geralmente com dor intensa na parte inferior direita do abdômen, podendo vir acompanhada de febre, náuseas e vômitos. _Emergência médica!_\n\n- **Cálculos biliares (pedra na vesícula)**  \n  > Dor na parte superior direita do abdô

In [ ]:
# Now let's check our Crew Response based into our input message
from IPython.display import display, Markdown

display(Markdown(resultado.raw))


# Laudo Médico Preliminar

---

## **Resumo dos Sintomas Apresentados**

Você relatou sentir **dor intensa no abdômen**, localizada ao _lado direito da barriga_. Este sintoma pode estar associado a diversas condições, algumas das quais exigem atenção médica imediata. Outros sintomas que podem estar presentes (mas não foram relatados) e que aumentam a gravidade incluem: febre, náuseas, vômitos, dificuldade para urinar, sangue na urina, icterícia (pele ou olhos amarelados) e alterações no hábito intestinal.

---

## **Possíveis Causas ou Diagnósticos Diferenciais**

Com base em informações médicas recentes e confiáveis, as principais causas para dor abdominal no lado direito incluem:

- **Apendicite**  
  > Inflamação do apêndice, geralmente com dor intensa na parte inferior direita do abdômen, podendo vir acompanhada de febre, náuseas e vômitos. _Emergência médica!_

- **Cálculos biliares (pedra na vesícula)**  
  > Dor na parte superior direita do abdômen, especialmente após refeições gordurosas.

- **Pedra nos rins**  
  > Dor que pode começar na região lombar e irradiar para o lado direito do abdômen, às vezes acompanhada de dificuldade para urinar ou sangue na urina.

- **Problemas no fígado**  
  > Hepatite ou acúmulo de gordura no fígado podem causar dor na parte superior direita da barriga, além de sintomas como cansaço, icterícia e alterações na cor das fezes e urina.

- **Síndrome do Intestino Irritável**  
  > Dores abdominais intermitentes, alterações no hábito intestinal (diarreia ou constipação).

- **Cisto ovariano (em mulheres)**  
  > Dor localizada, especialmente durante o ciclo menstrual, podendo causar inchaço abdominal ou irregularidades menstruais.

- **Tensão muscular**  
  > Lesões ou esforços físicos podem causar dor, geralmente menos grave, mas que pode limitar atividades.

---

## **Indicação de Medicamentos**

> **Atenção:** _Não é recomendado iniciar qualquer medicamento sem avaliação médica presencial, pois o tratamento depende da causa exata da dor._  
> Em situações de dor abdominal aguda, a automedicação pode mascarar sintomas importantes e atrasar o diagnóstico de condições graves, como apendicite.

---

## **Recomendações Clínicas e Orientações de Cuidado**

- **Procure atendimento médico imediatamente** se a dor for intensa, persistente, ou vier acompanhada de:
  - Febre
  - Vômitos
  - Dificuldade para urinar
  - Sangue na urina
  - Icterícia (pele ou olhos amarelados)
  - Alterações importantes nos hábitos intestinais

- **Evite automedicação.**  
  Não utilize analgésicos, anti-inflamatórios ou antibióticos sem prescrição médica.

- **Observe a evolução dos sintomas.**  
  Anote se surgirem novos sintomas ou se houver piora do quadro.

- **Em caso de piora súbita**, procure um pronto-socorro imediatamente, pois algumas causas (como apendicite) podem evoluir rapidamente e requerem intervenção urgente.

---

## **Observações Importantes**

> **Este laudo é preliminar e não substitui uma avaliação presencial com um médico.**  
> O diagnóstico definitivo depende de exame físico, histórico detalhado e, muitas vezes, exames laboratoriais e de imagem.

- _Evite esforços físicos até avaliação médica._
- _Mantenha-se hidratado, a menos que haja orientação médica contrária._
- _Se for mulher e houver atraso menstrual ou suspeita de gravidez, informe ao médico._

---

## **Fontes Utilizadas**

- Clínica Consulta: [Dor no lado direito da barriga: 7 causas (e o que fazer)](https://www.clinicaconsulta.com.br/dor-no-lado-direito-da-barriga-7-causas-e-o-que-fazer)
- MD.Saúde: [Dor abdominal: 40 causas e 13 sinais de gravidade](https://www.mdsaude.com/gastroenterologia/dor-na-barriga/)
- Passaporte da Saúde: [Dor do lado direito da barriga: principais causas e quando se preocupar](https://www.passaportedesaude.com.br/dor-do-lado-direito-da-barriga-principais-causas-e-quando-se-preocupar/)

---

> **Em caso de dúvida ou agravamento dos sintomas, procure atendimento médico imediatamente. Sua saúde é prioridade!**